# Autonomous driving - Car detection
Ideas in this notebook are from the two YOLO papers: [Redmon et al., 2016](https://arxiv.org/abs/1506.02640) and [Redmon and Farhadi, 2016](https://arxiv.org/abs/1612.08242).



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

In [ ]:
os.chdir('/content/drive/MyDrive/Datasets/Autonomous_car_driving/Car detection for Autonomous Driving')
#os.chdir('Car detection for Autonomous Driving')
#os.getcwd()
os.listdir()

['yolov4.weights',
 'Autonomous_driving_application_Car_detection_v3a.ipynb',
 'Drive.ai Dataset Sample LICENSE',
 'LICENSE',
 'font',
 'images',
 'model_data',
 'nb_images',
 'out',
 'yad2k',
 '__pycache__',
 '.ipynb_checkpoints',
 'yolo_utils.py',
 'yolov2.weights']

**Downloading YOLOv2 weights**

In [ ]:
!wget 'https://pjreddie.com/media/files/yolov2.weights'

--2021-05-07 02:42:39--  https://pjreddie.com/media/files/yolov2.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203934260 (194M) [application/octet-stream]
Saving to: ‘yolov2.weights’

yolov2.weights      100%[===================>] 194.49M  73.0MB/s    in 2.7s    

2021-05-07 02:42:41 (73.0 MB/s) - ‘yolov2.weights’ saved [203934260/203934260]



In [ ]:
#pip install --upgrade tensorflow-gpu==1.15

In [ ]:
#pip install --upgrade keras==2.3.1

**Downloading YOLOv4 pretrained weights..**

In [ ]:
#!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

Now we are in proper dir.

In [ ]:
#! git clone 'https://github.com/amanchadha/coursera-deep-learning-specialization.git'

In [ ]:
# import colorsys
# import imghdr
# import os
# import random
# from keras import backend as K

# import numpy as np
# from PIL import Image, ImageDraw, ImageFont

# def read_classes(classes_path):
#     with open(classes_path) as f:
#         class_names = f.readlines()
#     class_names = [c.strip() for c in class_names]
#     return class_names

# def read_anchors(anchors_path):
#     with open(anchors_path) as f:
#         anchors = f.readline()
#         try:
#           anchors = [float(x) for x in anchors.split(',')]
#         except ValueError as e:
#           print ("error",e,"on line")
#         anchors = np.array(anchors).reshape(-1, 2)
#     return anchors

# def generate_colors(class_names):
#     hsv_tuples = [(x / len(class_names), 1., 1.) for x in range(len(class_names))]
#     colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
#     colors = list(map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)), colors))
#     random.seed(10101)  # Fixed seed for consistent colors across runs.
#     random.shuffle(colors)  # Shuffle colors to decorrelate adjacent classes.
#     random.seed(None)  # Reset seed to default.
#     return colors

# def scale_boxes(boxes, image_shape):
#     """ Scales the predicted boxes in order to be drawable on the image"""
#     height = image_shape[0]
#     width = image_shape[1]
#     image_dims = K.stack([height, width, height, width])
#     image_dims = K.reshape(image_dims, [1, 4]).toString()
#     boxes = boxes * image_dims
#     return boxes

# def preprocess_image(img_path, model_image_size):
#     image_type = imghdr.what(img_path)
#     image = Image.open(img_path)
#     resized_image = image.resize(tuple(reversed(model_image_size)), Image.BICUBIC)
#     image_data = np.array(resized_image, dtype='float32')
#     image_data /= 255.
#     image_data = np.expand_dims(image_data, 0)  # Add batch dimension.
#     return image, image_data

# def draw_boxes(image, out_scores, out_boxes, out_classes, class_names, colors):

#     font = ImageFont.truetype(font='font/FiraMono-Medium.otf',size=np.floor(3e-2 * image.size[1] + 0.5).astype('int32'))
#     thickness = (image.size[0] + image.size[1]) // 300

#     for i, c in reversed(list(enumerate(out_classes))):
#         predicted_class = class_names[c]
#         box = out_boxes[i]
#         score = out_scores[i]

#         label = '{} {:.2f}'.format(predicted_class, score)

#         draw = ImageDraw.Draw(image)
#         label_size = draw.textsize(label, font)

#         top, left, bottom, right = box
#         top = max(0, np.floor(top + 0.5).astype('int32'))
#         left = max(0, np.floor(left + 0.5).astype('int32'))
#         bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
#         right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
#         print(label, (left, top), (right, bottom))

#         if top - label_size[1] >= 0:
#             text_origin = np.array([left, top - label_size[1]])
#         else:
#             text_origin = np.array([left, top + 1])

#         # My kingdom for a good redistributable image drawing library.
#         for i in range(thickness):
#             draw.rectangle([left + i, top + i, right - i, bottom - i], outline=colors[c])
#         draw.rectangle([tuple(text_origin), tuple(text_origin + label_size)], fill=colors[c])
#         draw.text(text_origin, label, fill=(0, 0, 0), font=font)
#         del draw

## Import libraries


In [ ]:
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
from PIL import Image
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Lambda, Conv2D
from keras.models import load_model, Model
from yolo_utils import read_classes, read_anchors, generate_colors, draw_boxes, scale_boxes,preprocess_image
from yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body

%matplotlib inline

In [ ]:
# yolo filter boxes

def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = .6):
    """Filters YOLO boxes by thresholding on object and class confidence.

    Arguments:
    box_confidence -- tensor of shape (19, 19, 5, 1)
    boxes -- tensor of shape (19, 19, 5, 4)
    box_class_probs -- tensor of shape (19, 19, 5, 80)
    threshold -- real value, if [ highest class probability score < threshold], then get rid of the corresponding box

    Returns:
    scores -- tensor of shape (None,), containing the class probability score for selected boxes
    boxes -- tensor of shape (None, 4), containing (b_x, b_y, b_h, b_w) coordinates of selected boxes
    classes -- tensor of shape (None,), containing the index of the class detected by the selected boxes

    """

    # Compute box scores
    box_scores = box_confidence * box_class_probs


    #  Finding the box_classes using the max box_scores
    box_classes = K.argmax(box_scores, axis = -1) # class having maximum box score
    box_class_scores = K.max(box_scores, axis= -1) # finding the box score of that max one.


    # Creating a filtering mask based on "box_class_scores" by using "threshold". The mask should have the
    # same dimension as box_class_scores, and be True for the boxes you want to keep (with probability >= threshold)
    filtering_mask = (box_class_scores >= threshold)


    # Applying the mask to box_class_scores, boxes and box_classes
    scores = tf.boolean_mask(box_class_scores, filtering_mask)
    boxes = tf.boolean_mask(boxes, filtering_mask)
    classes = tf.boolean_mask(box_classes, filtering_mask)

    return scores, boxes, classes

In [ ]:
with tf.Session() as test_a:
    box_confidence = tf.random_normal([19, 19, 5, 1], mean=1, stddev=4, seed = 1)
    boxes = tf.random_normal([19, 19, 5, 4], mean=1, stddev=4, seed = 1)
    box_class_probs = tf.random_normal([19, 19, 5, 80], mean=1, stddev=4, seed = 1)
    scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = 0.5)
    print("scores[2] = " + str(scores[2].eval()))
    print("boxes[2] = " + str(boxes[2].eval()))
    print("classes[2] = " + str(classes[2].eval()))
    print("scores.shape = " + str(scores.shape))
    print("boxes.shape = " + str(boxes.shape))
    print("classes.shape = " + str(classes.shape))

scores[2] = 10.750582
boxes[2] = [ 8.426533   3.2713668 -0.5313436 -4.9413733]
classes[2] = 7
scores.shape = (?,)
boxes.shape = (?, 4)
classes.shape = (?,)


In [ ]:
# Calculating iou(IoU) between box1 and box2


def iou(box1, box2):
    """
    Arguments:
    box1 -- first box, list object with coordinates (box1_x1, box1_y1, box1_x2, box_1_y2)
    box2 -- second box, list object with coordinates (box2_x1, box2_y1, box2_x2, box2_y2)
    """

    # Assigning variable names to coordinates for clarity
    (box1_x1, box1_y1, box1_x2, box1_y2) = box1
    (box2_x1, box2_y1, box2_x2, box2_y2) = box2
    #print(box1_x1, box1_y1, box1_x2, box1_y2)
    #print(box2_x1, box2_y1, box2_x2, box2_y2)

    # Calculating the (yi1, xi1, yi2, xi2) coordinates of the intersection of box1 and box2. Calculate its Area.
    xi1 = max(box1[0], box2[0])
    yi1 = max(box1[1], box2[1])
    xi2 = min(box1[2], box2[2])
    yi2 = min(box1[3], box2[3])


    inter_width = (xi2 - xi1)
    inter_height = (yi2 - yi1)
    inter_area = np.maximum((xi2 - xi1), 0)*np.maximum((yi2 - yi1),0)
    print("inter_area:", inter_area)


    # Union area by using Formula: Union(A,B) = A + B - Inter(A,B)
    box1_area = (box1_y2 - box1_y1) * (box1_x2 - box1_x1)
    box2_area = (box2_y2 - box2_y1) * (box2_x2 - box2_x1)
    union_area = (box1_area + box2_area) - inter_area

    # compute the IoU
    iou = inter_area / union_area

    return iou

In [ ]:
## Test case 1: boxes intersect
box1 = (2, 1, 4, 3)
box2 = (1, 2, 3, 4)
print("iou for intersecting boxes = " + str(iou(box1, box2)))

## Test case 2: boxes do not intersect
box1 = (1,2,3,4)
box2 = (5,6,7,8)
print("iou for non-intersecting boxes = " + str(iou(box1,box2)))

## Test case 3: boxes intersect at vertices only
box1 = (1,1,2,2)
box2 = (2,2,3,3)
print("iou for boxes that only touch at vertices = " + str(iou(box1,box2)))

## Test case 4: boxes intersect at edge only
box1 = (1,1,3,3)
box2 = (2,3,3,4)
print("iou for boxes that only touch at edges = " + str(iou(box1,box2)))

inter_area: 1
iou for intersecting boxes = 0.14285714285714285
inter_area: 0
iou for non-intersecting boxes = 0.0
inter_area: 0
iou for boxes that only touch at vertices = 0.0
inter_area: 0
iou for boxes that only touch at edges = 0.0


In [ ]:
# Applying yolo_non_max_suppression to set of boxes

def yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold = 0.5):
    """
    Arguments:
    scores -- tensor of shape (None,), output of yolo_filter_boxes()
    boxes -- tensor of shape (None, 4), output of yolo_filter_boxes() that have been scaled to the image size (see later)
    classes -- tensor of shape (None,), output of yolo_filter_boxes()
    max_boxes -- integer, maximum number of predicted boxes you'd like
    iou_threshold -- real value, "intersection over union" threshold used for NMS filtering

    Returns:
    scores -- tensor of shape (, None), predicted score for each box
    boxes -- tensor of shape (4, None), predicted box coordinates
    classes -- tensor of shape (, None), predicted class for each box
    """

    max_boxes_tensor = K.variable(max_boxes, dtype='int32')     # tensor to be used in tf.image.non_max_suppression()
    K.get_session().run(tf.variables_initializer([max_boxes_tensor])) # initialize variable max_boxes_tensor


    nms_indices = tf.image.non_max_suppression(boxes, scores, max_boxes_tensor, iou_threshold=iou_threshold)

    scores = K.gather(scores, nms_indices)
    boxes = K.gather(boxes, nms_indices)
    classes = K.gather(classes, nms_indices)


    return scores, boxes, classes

In [ ]:
with tf.Session() as test_b:
    scores = tf.random_normal([54,], mean=1, stddev=4, seed = 1)
    boxes = tf.random_normal([54, 4], mean=1, stddev=4, seed = 1)
    classes = tf.random_normal([54,], mean=1, stddev=4, seed = 1)
    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes)
    print("scores[2] = " + str(scores[2].eval()))
    print("boxes[2] = " + str(boxes[2].eval()))
    print("classes[2] = " + str(classes[2].eval()))
    print("scores.shape = " + str(scores.eval().shape))
    print("boxes.shape = " + str(boxes.eval().shape))
    print("classes.shape = " + str(classes.eval().shape))

scores[2] = 6.938395
boxes[2] = [-5.299932    3.1379814   4.450367    0.95942086]
classes[2] = -2.2452729
scores.shape = (10,)
boxes.shape = (10, 4)
classes.shape = (10,)


In [ ]:
# yolo eval

def yolo_eval(yolo_outputs, image_shape = (720., 1280.), max_boxes=10, score_threshold=.6, iou_threshold=.5):
    """
    Arguments:
    yolo_outputs -- output of the encoding model (for image_shape of (608, 608, 3)), contains 4 tensors:
                    box_confidence: tensor of shape (None, 19, 19, 5, 1)
                    box_xy: tensor of shape (None, 19, 19, 5, 2)
                    box_wh: tensor of shape (None, 19, 19, 5, 2)
                    box_class_probs: tensor of shape (None, 19, 19, 5, 80)
    image_shape -- tensor of shape (2,) containing the input shape, in this notebook we use (608., 608.) (has to be float32 dtype)
    max_boxes -- integer, maximum number of predicted boxes you'd like
    score_threshold -- real value, if [ highest class probability score < threshold], then get rid of the corresponding box
    iou_threshold -- real value, "intersection over union" threshold used for NMS filtering

    Returns:
    scores -- tensor of shape (None, ), predicted score for each box
    boxes -- tensor of shape (None, 4), predicted box coordinates
    classes -- tensor of shape (None,), predicted class for each box
    """

    # Retrieve outputs of the YOLO model
    box_confidence, box_xy, box_wh, box_class_probs = yolo_outputs

    # Converting boxes to be ready for filtering functions (convert boxes box_xy and box_wh to corner coordinates)
    boxes = yolo_boxes_to_corners(box_xy, box_wh)

    scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = score_threshold)

    # Scale boxes back to original image shape.
    boxes = scale_boxes(boxes, image_shape)

    # Use one of the functions you've implemented to perform Non-max suppression with
    # maximum number of boxes set to max_boxes and a threshold of iou_threshold
    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes, max_boxes,iou_threshold)

    return scores, boxes, classes

In [ ]:
with tf.Session() as test_b:
    yolo_outputs = (tf.random_normal([19, 19, 5, 1], mean=1, stddev=4, seed = 1),
                    tf.random_normal([19, 19, 5, 2], mean=1, stddev=4, seed = 1),
                    tf.random_normal([19, 19, 5, 2], mean=1, stddev=4, seed = 1),
                    tf.random_normal([19, 19, 5, 80], mean=1, stddev=4, seed = 1))
    scores, boxes, classes = yolo_eval(yolo_outputs)
    print("scores[2] = " + str(scores[2].eval()))
    print("boxes[2] = " + str(boxes[2].eval()))
    print("classes[2] = " + str(classes[2].eval()))
    print("scores.shape = " + str(scores.eval().shape))
    print("boxes.shape = " + str(boxes.eval().shape))
    print("classes.shape = " + str(classes.eval().shape))

scores[2] = 138.79124
boxes[2] = [1292.3297  -278.52167 3876.9893  -835.56494]
classes[2] = 54
scores.shape = (10,)
boxes.shape = (10, 4)
classes.shape = (10,)


##  Test YOLO pre-trained model on images

Here we gonna use a pre-trained model and test it on the car detection dataset.  We'll need a session to execute the computation graph and evaluate the tensors.

In [ ]:
sess = K.get_session()

**Doing some stuffs**

In [ ]:
# # importing required modules
# from zipfile import ZipFile

# # specifying the zip file name
# file_name = "/content/drive/MyDrive/Datasets/Autonomous_car_driving/Car detection for Autonomous Driving/yolo.h5.zip"

# # opening the zip file in READ mode
# with ZipFile(file_name, 'r') as zip:
#     # printing all the contents of the zip file
#     zip.printdir()

#     # extracting all the files
#     print('Extracting all the files now...')
#     zip.extractall()
#     print('Done!')

In [ ]:
# def fixBadZipfile(zipFile):
#   f = open(zipFile, 'r+b')
#   data = f.read()
#   pos = data.find('\x50\x4b\x05\x06') # End of central directory signature
#   if (pos > 0):
#       self._log("Trancating file at location " + str(pos + 22)+ ".")
#       f.seek(pos + 22)   # size of 'ZIP end of central directory record'
#       f.truncate()
#       f.close()


In [ ]:
class_names = read_classes("model_data/coco_classes.txt")
anchors = read_anchors("model_data/yolo_anchors.txt")
image_shape = (720., 1280.)

###  Loading a pre-trained model

In [ ]:
yolo_model = load_model("model_data/yolo.h5")

/usr/local/lib/python3.7/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [ ]:
import keras
opt = keras.optimizers.Adam(learning_rate=0.01)

In [ ]:
yolo_model.compile(loss='categorical_crossentropy', optimizer=opt,  weighted_metrics= '/content/drive/MyDrive/Datasets/Autonomous_car_driving/Car detection for Autonomous Driving/yolov2.weights')

TypeError: ignored

This loads the weights of a trained YOLO model. Here's a summary of the layers your model contains.

In [ ]:
yolo_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 608, 608, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 608, 608, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 608, 608, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 608, 608, 32) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

###  Convert output of the model to usable bounding box tensors

The output of `yolo_model` is a (m, 19, 19, 5, 85) tensor that needs to pass through non-trivial processing and conversion.

 `yolo_head` implementation can find the function definition in the file ['keras_yolo.py'](https://github.com/allanzelener/YAD2K/blob/master/yad2k/models/keras_yolo.py).

In [ ]:
yolo_outputs = yolo_head(yolo_model.output, anchors, len(class_names))

###  Filtering boxes

`yolo_outputs` gave us all the predicted boxes of `yolo_model` in the correct format.

In [ ]:
scores, boxes, classes = yolo_eval(yolo_outputs, image_shape)

### Running the graph on an image


In [ ]:
def predict(sess, image_file):
    """
    Runs the graph stored in "sess" to predict boxes for "image_file". Prints and plots the predictions.

    Arguments:
    sess -- your tensorflow/Keras session containing the YOLO graph
    image_file -- name of an image stored in the "images" folder.

    Returns:
    out_scores -- tensor of shape (None, ), scores of the predicted boxes
    out_boxes -- tensor of shape (None, 4), coordinates of the predicted boxes
    out_classes -- tensor of shape (None, ), class index of the predicted boxes

    Note: "None" actually represents the number of predicted boxes, it varies between 0 and max_boxes.
    """

    # Preprocess your image
    print('line 1')
    image, image_data = preprocess_image("images/" + image_file, model_image_size = (608, 608))
    print(image_data.shape)

    # Run the session with the correct tensors and choose the correct placeholders in the feed_dict.
    # You'll need to use feed_dict={yolo_model.input: ... , K.learning_phase(): 0})
    ### START CODE HERE ### (≈ 1 line)
    print('line 2')
    out_scores, out_boxes, out_classes = sess.run([scores, boxes, classes], feed_dict={yolo_model.input: image_data, K.learning_phase():0})
    ### END CODE HERE ###

    # Print predictions info
    print('Found {} boxes for {}'.format(len(out_boxes), image_file))
    # Generate colors for drawing bounding boxes.
    colors = generate_colors(class_names)
    # Draw bounding boxes on the image file
    draw_boxes(image, out_scores, out_boxes, out_classes, class_names, colors)
    # Save the predicted bounding box on the image
    image.save(os.path.join("out", image_file), quality=90)
    # Display the results in the notebook
    output_image = scipy.misc.imread(os.path.join("out", image_file))
    imshow(output_image)

    return out_scores, out_boxes, out_classes

In [ ]:
out_scores, out_boxes, out_classes = predict(sess, "test.jpg")

**References**: The ideas presented in this notebook came primarily from the two YOLO papers. The implementation here also took significant inspiration and used many components from Allan Zelener's GitHub repository. The pre-trained weights used in this exercise came from the official YOLO website.
- Joseph Redmon, Santosh Divvala, Ross Girshick, Ali Farhadi - [You Only Look Once: Unified, Real-Time Object Detection](https://arxiv.org/abs/1506.02640) (2015)
- Joseph Redmon, Ali Farhadi - [YOLO9000: Better, Faster, Stronger](https://arxiv.org/abs/1612.08242) (2016)
- Allan Zelener - [YAD2K: Yet Another Darknet 2 Keras](https://github.com/allanzelener/YAD2K)
- The official YOLO website (https://pjreddie.com/darknet/yolo/)